<a href="https://colab.research.google.com/github/ckhsmile/ds2_deep_learning_hw/blob/main/hw_fnnae_creditcard_skeleton_DS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Credit Card Fraud Detection

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
#os.chdir("{fill in your path}")
os.chdir("/content/drive/MyDrive/Data_Scientist_과정(11기)/12_DL_강유 교수님/03 프로젝트")
# os.chdir("/content/drive/MyDrive/Colab Notebooks/2023-03-DS2/hw-fnnae-creditcard")

### Load Data

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras.backend as K # 백엔드 모듈, 모델을 정의하고 학습할 때 텐서 연산 등을 수행하는 데 사용

train = pd.read_csv('data/creditcard_train.csv')
test = pd.read_csv('data/creditcard_test.csv')

In [ ]:
train.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,123078.0,-1.073820,0.415616,-2.273977,1.536844,-0.758697,-1.670381,-2.377140,0.090370,0.004847,...,0.535542,0.863592,0.450743,-0.144228,-0.205609,-0.539073,0.503418,-0.237807,11.0,1
1,35899.0,-2.857170,4.045601,-4.197299,5.487199,-3.070776,-1.422686,-5.651314,2.019657,-5.015491,...,1.080323,-0.561384,0.102678,-0.067195,-0.476931,-0.103716,1.166961,0.663632,1.0,1
2,94364.0,-15.192064,10.432528,-19.629515,8.046075,-12.838167,-1.875859,-21.359738,-3.717850,-5.969782,...,-3.474097,1.765446,1.701257,0.381587,-1.413417,-1.023078,-2.634761,-0.463931,1.0,1
3,42988.0,-4.423508,1.648048,-6.934388,4.894601,-5.078131,0.010849,-3.409096,1.409291,-3.260672,...,0.698359,0.487478,1.228698,-0.535217,0.388278,-0.009466,2.300164,0.081231,648.0,1
4,8408.0,-1.813280,4.917851,-5.926130,5.701500,1.204393,-3.035138,-1.713402,0.561257,-3.796354,...,0.615642,-0.406427,-0.737018,-0.279642,1.106766,0.323885,0.894767,0.569519,1.0,1


In [ ]:
train.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,227846.000000,227846.000000,227846.000000,227846.000000,227846.000000,227846.000000,227846.000000,227846.000000,227846.000000,227846.000000,...,227846.000000,227846.000000,227846.000000,227846.000000,227846.000000,227846.000000,227846.000000,227846.000000,227846.000000,227846.000000
mean,94752.688750,-0.003248,-0.001924,0.001174,-0.000395,0.000991,0.000627,0.000272,0.002218,-0.000261,...,0.000649,0.001206,-0.001100,0.000270,0.000243,-0.001124,-0.000259,0.000463,88.518239,0.001729
std,47501.831555,1.961936,1.660071,1.515244,1.414744,1.366611,1.325355,1.214064,1.187842,1.096247,...,0.731021,0.724135,0.625186,0.605983,0.521350,0.482339,0.399823,0.331192,248.088506,0.041548
min,0.000000,-56.407510,-72.715728,-32.965346,-5.683171,-42.147898,-26.160506,-43.557242,-73.216718,-13.320155,...,-34.830382,-10.933144,-44.807735,-2.836627,-10.295397,-2.604551,-22.565679,-11.710896,0.000000,0.000000
25%,54182.000000,-0.922796,-0.598069,-0.889210,-0.848880,-0.690669,-0.767855,-0.554761,-0.207819,-0.643382,...,-0.228018,-0.540724,-0.162271,-0.354105,-0.317437,-0.327907,-0.070990,-0.053115,5.580000,0.000000
50%,84608.000000,0.012768,0.066598,0.182150,-0.019309,-0.055161,-0.273090,0.040411,0.022960,-0.050980,...,-0.028794,0.008684,-0.011626,0.041212,0.016272,-0.053256,0.001314,0.011216,22.000000,0.000000
75%,139342.500000,1.314872,0.804387,1.029494,0.744851,0.610928,0.400138,0.570642,0.327863,0.596617,...,0.186852,0.529495,0.147076,0.440035,0.351245,0.239848,0.091097,0.078460,77.080000,0.000000
max,172792.000000,2.454930,22.057729,9.382558,16.875344,34.801666,22.529298,36.877368,20.007208,15.594995,...,27.202839,10.503090,22.528412,4.022866,6.070850,3.463246,12.152401,33.847808,19656.530000,1.000000


In [ ]:
# null 값 확인
train.isnull().sum().max()

0

In [ ]:
train.columns

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')

In [ ]:
test.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99
1,10.0,1.449044,-1.176339,0.913860,-1.375667,-1.971383,-0.629152,-1.423236,0.048456,-1.720408,...,-0.387226,-0.009302,0.313894,0.027740,0.500512,0.251367,-0.129478,0.042850,0.016253,7.80
2,17.0,0.962496,0.328461,-0.171479,2.109204,1.129566,1.696038,0.107712,0.521502,-1.191311,...,-0.269321,0.143997,0.402492,-0.048508,-1.371866,0.390814,0.199964,0.016371,-0.014605,34.09
3,23.0,1.322707,-0.174041,0.434555,0.576038,-0.836758,-0.831083,-0.264905,-0.220982,-1.071425,...,-0.522951,-0.284376,-0.323357,-0.037710,0.347151,0.559639,-0.280158,0.042335,0.028822,16.00
4,26.0,-0.535388,0.865268,1.351076,0.147575,0.433680,0.086983,0.693039,0.179742,-0.285642,...,-0.283264,0.049526,0.206537,-0.187108,0.000753,0.098117,-0.553471,-0.078306,0.025427,1.77


불균형 데이터 참고 부분  
*   https://www.tensorflow.org/tutorials/structured_data/imbalanced_data?hl=ko  

In [ ]:
normal, fraud = np.bincount(train['Class'])
total = normal + fraud
print('Data:\n    Total: {}\n    Normal: {} ({:.3f}% of total)\n    Fraud: {} ({:.3f}% of total)\n'.format(
    total, normal, 100 * normal / total, fraud, 100 * fraud / total))

Data:
    Total: 227846
    Normal: 227452 (99.827% of total)
    Fraud: 394 (0.173% of total)



### 데이터 정리, 분할 및 정규화

`Time` 및 `Amount` 열이 매우 가변적이므로 직접 사용할 수 없음. (의미가 명확하지 않으므로)  
`Time` 열을 삭제하고 `Amount` 열의 로그를 가져와 범위를 줄이기

In [ ]:
cleaned_train = train.copy()
cleaned_test = test.copy()

# You don't want the `Time` column.
cleaned_train.pop('Time')
cleaned_test.pop('Time')

# The `Amount` column covers a huge range. Convert to log-space.
eps = 0.001 # 0 => 0.1¢
cleaned_train['Log Amount'] = np.log(cleaned_train.pop('Amount')+eps)
cleaned_test['Log Amount'] = np.log(cleaned_test.pop('Amount')+eps)

In [ ]:
train = cleaned_train.copy()
test = cleaned_test.copy()

In [ ]:
# oversampling
# normal과 fraud class의 비율 차이가 크므로, 올바른 알고리즘 training을 위해 동일한 개수를 가지도록 oversampling

from imblearn.over_sampling import RandomOverSampler, SMOTE
# RandomOverSampler는 소수 클래스 데이터를 무작위로 복제하기 때문에,
# 데이터의 양을 크게 늘릴 수 있다는 장점이 있으나, 무작위로 복제하기 때문에 과적합이 발생할 수 있다는 단점이 있음.
# SMOTE는 소수 클래스 데이터 주변의 다수 클래스 데이터를 사용하여 새로운 샘플을 생성하기 때문에, 소
# 소수 클래스 데이터의 특성을 잘 보존할 수 있다는 장점이 있으나, 데이터의 특성이 왜곡될 수 있다는 단점이 있음.

fraud_df = train[train.Class==1]
normal = train[train.Class==0]

print('Before Sampling:')
print(f'  (Original) # of Normal(Class = 0): {normal.shape[0]}')
print(f'  (Original) # of Fraud(Class = 1): {fraud_df.shape[0]}')

train, y = SMOTE(random_state=0).fit_resample(train.drop(columns='Class', axis=0), train['Class'])
# train, y = RandomOverSampler(random_state=0).fit_resample(train.drop(columns='Class', axis=0), train['Class'])
train['Class'] = y

num_fraud = train[train['Class']==1].Class.count()
num_normal = train[train['Class']==0].Class.count()

print('\nAfter Resampling:')
print(f'  (Balanced) Number of Normal (Class=0): {num_normal}')
print(f'  (Balanced) Number of Fraud (Class=1): {num_fraud}')

Before Sampling:
  (Original) # of Normal(Class = 0): 227452
  (Original) # of Fraud(Class = 1): 394

After Resampling:
  (Balanced) Number of Normal (Class=0): 227452
  (Balanced) Number of Fraud (Class=1): 227452


### Split dataset into train and test

In [ ]:
# Use `shuffle` function from sklearn.utils
# Shuffling the sorted data can increase performances of model
from sklearn.utils import shuffle

X_train = shuffle(train, random_state=0)
X_test = test

In [ ]:
normal, fraud = np.bincount(X_train['Class'])
total = normal + fraud
print('Data:\n    Total: {}\n    Normal: {} ({:.3f}% of total)\n    Fraud: {} ({:.3f}% of total)\n'.format(
    total, normal, 100 * normal / total, fraud, 100 * fraud / total))

Data:
    Total: 454904
    Normal: 227452 (50.000% of total)
    Fraud: 227452 (50.000% of total)



In [ ]:
# Extract the label and Perform the one-hot encoding
y_train = X_train['Class'].values.astype('float')
y_train = np.stack([1 - y_train, y_train]).T # One-hot encoding
y_train.shape

(454904, 2)

### Normalize data

In [ ]:
fields = ["Class"]
X_train = X_train.drop(fields, axis=1)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train) # fit_transform은 학습 데이터에 대해서만 사용. 모델을 학습하여 데이터의 특징과 분포를 이해하고, 이에 따라 데이터를 변환
X_test = scaler.transform(X_test) # 새로운 데이터를 기존의 학습한 스케일링 모델에 맞게 변환

### Define the model

In [ ]:
print(X_train.shape)
print(y_train.shape)

(454904, 29)
(454904, 2)


In [ ]:
import tensorflow
from keras.layers import Dense, Dropout
from keras.models import Sequential # type of model

In [ ]:
model = Sequential([
    tf.keras.Input(shape=X_train.shape[1]),
    Dense(64, activation = 'relu'),
    Dense(32, activation = 'relu'),
    Dense(16, activation = 'relu'),
    Dropout(0.5),
    Dense(2, activation="sigmoid") # softmax
  ]
)


In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 64)                1920      
                                                                 
 dense_5 (Dense)             (None, 32)                2080      
                                                                 
 dense_6 (Dense)             (None, 16)                528       
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense_7 (Dense)             (None, 2)                 34        
                                                                 
Total params: 4,562
Trainable params: 4,562
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def recall(y_true, y_pred):
    y_true = K.argmax(y_true, axis=-1)
    y_pred = K.argmax(y_pred, axis=-1)
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)
    predicted_positives = tf.tensordot(K.cast(y_true, dtype=tf.float32), K.cast(y_pred, dtype=tf.float32), axes=1)
    true_positives = K.sum(K.cast(y_true, dtype=tf.float32))
    recall_keras = predicted_positives / (true_positives + K.epsilon())
    return recall_keras

In [ ]:
model.compile(optimizer = 'adam',
              loss = 'categorical_crossentropy',
              metrics = [recall])

In [ ]:
EPOCHS = 100
BATCH_SIZE = 2048

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_recall',
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)

In [ ]:
# model.fit(X_train, y_train, batch_size= BATCH_SIZE, epochs = EPOCHS, validation_split = 0.15, callbacks=[early_stopping])
# model.fit(X_train, y_train, batch_size= BATCH_SIZE, epochs = EPOCHS, validation_split = 0.15, verbose = 1)
# verbose = 0: silent, verbose = 1: progress bar, verbose = 2: one line per epoch

Epoch 1/100
189/189 [==============================] - 4s 20ms/step - loss: 0.0033 - recall: 0.9999 - val_loss: 0.0036 - val_recall: 1.0000
Epoch 2/100
189/189 [==============================] - 2s 13ms/step - loss: 0.0039 - recall: 0.9999 - val_loss: 0.0019 - val_recall: 1.0000
Epoch 3/100
189/189 [==============================] - 3s 13ms/step - loss: 0.0036 - recall: 0.9999 - val_loss: 0.0026 - val_recall: 1.0000
Epoch 4/100
189/189 [==============================] - 1s 8ms/step - loss: 0.0034 - recall: 0.9999 - val_loss: 0.0025 - val_recall: 1.0000
Epoch 5/100
189/189 [==============================] - 1s 8ms/step - loss: 0.0036 - recall: 0.9999 - val_loss: 0.0028 - val_recall: 0.9997
Epoch 6/100
189/189 [==============================] - 1s 8ms/step - loss: 0.0033 - recall: 0.9999 - val_loss: 0.0016 - val_recall: 1.0000
Epoch 7/100
189/189 [==============================] - 2s 11ms/step - loss: 0.0034 - recall: 0.9999 - val_loss: 0.0035 - val_recall: 1.0000
Epoch 8/100
189/189 [==

In [ ]:
# Evaluate the peformance using the training dataset (loss, precision, recall, binary_accuracy)
score = model.evaluate(X_train, y_train, verbose=0)
print(score)
print("Train loss:", score[0])
print("Train recall:", score[1])

[0.0009491804521530867, 1.0]
Train loss: 0.0009491804521530867
Train recall: 1.0


In [ ]:
from keras.metrics import Recall
from keras.metrics import Precision
from keras.metrics import BinaryAccuracy
from keras.metrics import AUC

model.compile(optimizer = 'adam',
              loss = 'binary_crossentropy',
              metrics = [Precision(name='precision')\
                        ,Recall(name='recall')\
                        ,BinaryAccuracy()\
                        ,AUC()])

In [ ]:
EPOCHS = 100
BATCH_SIZE = 2048

model.fit(X_train, y_train, batch_size= BATCH_SIZE, epochs = EPOCHS, validation_split = 0.2, verbose=1)
# model.fit(X_train, y_train, batch_size = 100, epochs = 50, validation_split = 0.15, verbose=1)

Epoch 1/100
178/178 [==============================] - 3s 10ms/step - loss: 0.3531 - precision: 0.8174 - recall: 0.8534 - binary_accuracy: 0.8314 - auc_2: 0.9243 - val_loss: 0.1196 - val_precision: 0.9503 - val_recall: 0.9538 - val_binary_accuracy: 0.9520 - val_auc_2: 0.9924
Epoch 2/100
178/178 [==============================] - 1s 8ms/step - loss: 0.1190 - precision: 0.9545 - recall: 0.9497 - binary_accuracy: 0.9522 - auc_2: 0.9919 - val_loss: 0.0438 - val_precision: 0.9853 - val_recall: 0.9873 - val_binary_accuracy: 0.9863 - val_auc_2: 0.9987
Epoch 3/100
178/178 [==============================] - 2s 13ms/step - loss: 0.0572 - precision: 0.9869 - recall: 0.9694 - binary_accuracy: 0.9783 - auc_2: 0.9981 - val_loss: 0.0196 - val_precision: 0.9952 - val_recall: 0.9962 - val_binary_accuracy: 0.9957 - val_auc_2: 0.9997
Epoch 4/100
178/178 [==============================] - 2s 14ms/step - loss: 0.0351 - precision: 0.9943 - recall: 0.9800 - binary_accuracy: 0.9872 - auc_2: 0.9992 - val_loss:

In [ ]:
# Evaluate the peformance using the training dataset (loss, precision, recall, binary_accuracy)
score = model.evaluate(X_train, y_train, verbose=0)
print("Train loss:", score[0])
print("Train precision:", score[1])
print("Train recall:", score[2])
print("Train binary_accuracy:", score[3])

Train loss: 0.0010474763112142682
Train precision: 0.9998593330383301
Train recall: 0.9998593330383301
Train binary_accuracy: 0.9998593330383301


In [ ]:
# Predict using the test data
y_predicted = model.predict(X_test)
y_test = [1 if y[0]<y[1] else 0 for y in y_predicted]
print(f'Total N = {len(y_test)}, Fraud N = {y_test.count(1)}, Normal N = {y_test.count(0)}')

1781/1781 [==============================] - 2s 1ms/step
Total N = 56961, Fraud N = 127, Normal N = 56834


In [ ]:
import csv

# with open("hw1_result.csv", "wb") as f:
#   csv_writer = csv.writer(f)

result = pd.read_csv('data/hw1_results.csv')
result['Class'] = y_test

result.to_csv('data/DS2_hw1_조경희5.csv', index=False)
